In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - XGBoost Model Experimentation

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/04_model_experimentation.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/04_model_experimentation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/04_model_experimentation.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to pull features from Feature Store for training, run data exploratory analysis on features, build a machine learning model locally, experiment with various hyperparameters, evaluate the model and deloy it to a Vertex AI endpoint. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

- Use a Feature Store to pull training data
- Do some exploratory analysis on the extracted data
- Train the model and track the results using Vertex AI Experiments

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Import libraries

In [ ]:
# General
import os
import sys
from typing import Union, List
import random
from datetime import datetime, timedelta
import time
import json
import logging

# Feature Store
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import Featurestore, EntityType, Feature

# Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Model Training
from google.cloud import bigquery
from google.cloud import storage
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

### Define constants

In [ ]:
# General
TIMESTAMP = str(int(time.time()))
DATA_DIR = os.path.join(os.pardir, "data")
DATA_URI = f"gs://{BUCKET_NAME}/data"
TRAIN_DATA_URI = f"{DATA_URI}/train"

# Feature Store
START_DATE_TRAIN = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
END_DATE_TRAIN = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
CUSTOMER_ENTITY = "customer"
TERMINAL_ENTITY = "terminal"
SERVING_FEATURE_IDS = {CUSTOMER_ENTITY: ["*"], TERMINAL_ENTITY: ["*"]}
READ_INSTANCES_TABLE = f"ground_truth_{END_DATE_TRAIN}"
READ_INSTANCES_URI = f"bq://{PROJECT_ID}.tx.{READ_INSTANCES_TABLE}"

# Training
COLUMNS_IGNORE = ["terminal_id", "customer_id", "entity_type_event", "entity_type_customer", "entity_type_terminal"]
TARGET = "tx_fraud"

# Custom Training
MODEL_NAME=f"fraudfinder_xgb_model_expr-{ID}-{TIMESTAMP}"

## Experiment
EXPERIMENT_NAME = "ff-experiment-" + TIMESTAMP

### Initialize clients
Next you have to initialize the [Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/use-vertex-ai-python-sdk) and the Python BigQuery Client for your project, region and corresponding bucket.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME, 
               experiment=EXPERIMENT_NAME)

### Helper Functions
Next you will create a helper function for sending queries to BigQuery.

In [ ]:
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

def create_gcs_dataset(client,
                       display_name: str,
                       gcs_source: Union[str, List[str]]):
    """
    A function to create a Vertex AI Dataset resource
    Args:
        client: VertexAI Client instance
        display_name: The name of Vertex AI Dataset resource
        gcs_source: The uri of data on the bucket
    Returns:
        VertexAI Dataset resource
    """
    dataset = client.TabularDataset.create(
        display_name=display_name, gcs_source=gcs_source,
    )

    dataset.wait()
    return dataset

def preprocess(df):
    """Converts categorical features to numeric. Removes unused columns.
    Args:
      df: Pandas df with raw data
    Returns:
      df with preprocessed data
    """
    df = df.drop(columns=UNUSED_COLUMNS)

    # Drop rows with NaN"s
    df = df.dropna()

    # Convert integer valued (numeric) columns to floating point
    numeric_columns = df.select_dtypes(["int32", "float32", "float64"]).columns
    df[numeric_columns] = df[numeric_columns].astype("float32")

    dummy_columns = list(df.dtypes[df.dtypes == "category"].index)
    df = pd.get_dummies(df, columns=dummy_columns)

    return df

## Fetching feature values for model training

To fetch training data, we have to specify the following inputs to batch serving:

- a file containing a "query", with the entities and timestamps for each label
- a list of features to fetch values for
- the destination location and format


### Read-instance list

In our case, we need a csv file with content formatted like the table below:

|customer                     |terminal|timestamp                                    |
|-----------------------------|--------|---------------------------------------------|
|xxx3859                         |xxx8811    |2021-07-07 00:01:10 UTC                      |
|xxx4165                         |xxx8810    |2021-07-07 00:01:55 UTC                      |
|xxx2289                         |xxx2081    |2021-07-07 00:02:12 UTC                      |
|xxx3227                         |xxx3011    |2021-07-07 00:03:23 UTC                      |
|xxx2819                         |xxx6263    |2021-07-07 00:05:30 UTC                      |

where the column names are the name of entities in Feature Store and the timestamps represents the time an event occurred.

In [ ]:
read_instances_query = f"""
    SELECT
        raw_tx.TX_TS AS timestamp,
        raw_tx.CUSTOMER_ID AS customer,
        raw_tx.TERMINAL_ID AS terminal,
        raw_tx.TX_AMOUNT AS tx_amount,
        raw_lb.TX_FRAUD AS tx_fraud,
    FROM 
        tx.tx as raw_tx
    LEFT JOIN 
        tx.txlabels as raw_lb
    ON raw_tx.TX_ID = raw_lb.TX_ID
    WHERE
        DATE(raw_tx.TX_TS) = "{START_DATE_TRAIN}";
"""
print(read_instances_query)

query_df = run_bq_query(read_instances_query)

Lets check our read instance list

In [ ]:
query_df.head(4)

### Get Feature Store
Now you can get the feature store ID so that we can fetch a batch of training data. 

In [ ]:
ff_feature_store = Featurestore(FEATURESTORE_ID)

### Export a training sample of data to a pandas dataframe.
First you need to fetch a batch of data. We will use this data to train a custom model. We will fetch a batch of data and create a Pandas dataframe. The dataframe makes it easier to inspect the data. 

In [ ]:
sample_df = ff_feature_store.batch_serve_to_df(
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_df=query_df,
    pass_through_fields = ['tx_fraud','tx_amount']
)

Let's have a look at the data. 

In [ ]:
sample_df.head(5)

### Data exploration
Here you will use a subset of data for data exploration to get a better understanding of the data. 

In [ ]:
sample_df.info()

#### Examine the label distibution
With Fraud detection we often have a challenge with the distribution of our target value. Fraud vs non-Fraud can be skewed. We want to understand the distribution. Lets create a plot and do an actual count of the values (fraud vs non-fraud).

In [ ]:
plt.title("fraud_reported distribution")
sns.countplot(x="tx_fraud", data=sample_df)

In [ ]:
(sample_df.tx_fraud.value_counts()/sample_df.shape[0])*100

As we can see the data is imbalanced. We will fix this later in this notebook before building the model.

#### Get a better understanding of the feature distributions using histograms
Now you can do the same for the input features. It's good to plot the distributions of our features to get a better understanding of them. 

In [ ]:
features = [ "tx_amount", "customer_id_nb_tx_7day_window", "customer_id_nb_tx_14day_window",
             "customer_id_nb_tx_1day_window", "customer_id_avg_amount_7day_window", "customer_id_avg_amount_14day_window",
             "customer_id_avg_amount_1day_window", "terminal_id_risk_7day_window", "terminal_id_risk_14day_window",
             "terminal_id_risk_1day_window", "terminal_id_nb_tx_14day_window", "terminal_id_nb_tx_1day_window",
            "terminal_id_nb_tx_7day_window"]

sample_df[features].hist(figsize = (20, 10), grid=False);

#### Analyze the relationship between features and ```tx_fraud```
Now you can also look at the relationship between our target and input features. A good way to do this is creating a correlation heatmap. 

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(sample_df[sample_df.columns.difference(COLUMNS_IGNORE)].corr())

**Some Observations**

Based on this simple exploratory data analysis (EDA) you can conclude:

- The sample data is unbalanced.
- You can probably remove some of the features with little predictive value.
- You might want to extract subsets of the timestamp into separate features such as day, week, night, etc. (i.e. calculate some time-based embeddings).
- You may want to do some variable selection.
- You'll might need to scale some variables.

## Builing a custom fraud detection model

### Fixing an imbalanced dataset
In the real world, we need to deal with imbalance in our dataset. For example, we might randomly delete some of the non-fraudulent transcations in order to approximately match the number of fraudulent transactions. This technique is called undersampling. 

For this workshop, we will skip the data balance process, because our sample data is small and further reduction will compromise the quality of our results. 

In [ ]:
shuffled_df = sample_df.sample(frac=1,random_state=4)
fraud_df = shuffled_df.loc[shuffled_df["tx_fraud"] == 1]
non_fraud_df = shuffled_df.loc[shuffled_df["tx_fraud"] == 0].sample(n=fraud_df.shape[0],random_state=42)
balanced_df = pd.concat([fraud_df, non_fraud_df])
(balanced_df.tx_fraud.value_counts()/balanced_df.shape[0])*100

### Training
In this section, we will train a model using the xgboost algorithm. Typically, to perform training, you might want to use a Vertex AI traning pipeline, however, as we are still experimenting, we simply use the xgboost package interactively to train our model in this notebook. 

#### Why XGBoost?
Extreme gradient-boosted (XGBoost) algorithm is a ML algorithm based on ensembles of decision trees, that works well with imbalanced data, handleing missing values, and can speed up through parallel-processing.

#### Preparing datasets
In the following cell, we split our data into training, test, and validation sets. Training data is the primary source of input for training the ML algorithm. Validation data is for determining our progress after each epoch or iteration of our training loop. Test data is data the model has never seen before and is used to assess model quality at the end of the training process.

In [ ]:
# Set up training variables
LABEL_COLUMN = "tx_fraud"
UNUSED_COLUMNS = ["timestamp","entity_type_customer","entity_type_terminal"]
NA_VALUES = ["NA", "."]

df_dataset = balanced_df
df_train, df_test, df_val = np.split(df_dataset.sample(frac=1, random_state=42), [int(.6*len(df_dataset)), int(.8*len(df_dataset))])

#training set
preprocessed_train_Data = preprocess(df_dataset)
x_train = preprocessed_train_Data[preprocessed_train_Data.columns.drop(LABEL_COLUMN).to_list()].values
y_train = preprocessed_train_Data.loc[:,LABEL_COLUMN].astype(int)

#validation set
preprocessed_val_Data = preprocess(df_val)
x_val = preprocessed_val_Data[preprocessed_val_Data.columns.drop(LABEL_COLUMN).to_list()].values
y_val = preprocessed_val_Data.loc[:,LABEL_COLUMN].astype(int)

#test set
preprocessed_test_Data = preprocess(df_test)
x_test = preprocessed_test_Data[preprocessed_test_Data.columns.drop(LABEL_COLUMN).to_list()].values
y_test = preprocessed_test_Data.loc[:,LABEL_COLUMN].astype(int)

#### Training the model
Before training the XGBoost model, you can set some of the hyperparameters, which can help us with improving the model performance. We would advise you to use [Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview) to help with hyperparameter tuning. However, in this notebook, for the sake of simplicity and expedience, we specify these hyperparemeters manually and randomly. 

- Eta: A regularization parameter to reduce feature weights in each boosting step. 
- Gamma: A regularization parameter for tree pruning.
- Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit. 


For more information about parameters check [this document](https://xgboost.readthedocs.io/en/stable/parameter.html).

In [ ]:
parameters = [
    {"eta": 0.2, "gamma": 0.0, "max_depth": 4},
    {"eta": 0.2, "gamma": 0.0, "max_depth": 5},
    {"eta": 0.2, "gamma": 0.1, "max_depth": 4},
    {"eta": 0.2, "gamma": 0.1, "max_depth": 5},
    {"eta": 0.3, "gamma": 0.0, "max_depth": 4},
    {"eta": 0.3, "gamma": 0.0, "max_depth": 5},
    {"eta": 0.3, "gamma": 0.1, "max_depth": 4},
    {"eta": 0.3, "gamma": 0.1, "max_depth": 5},
]

models={}
for i, params in enumerate(parameters):
    run_name=f"fd-xgboost-local-run-{i}"
    print(run_name)
    vertex_ai.start_run(run=run_name)
    vertex_ai.log_params(params)
    model =  xgb.XGBClassifier( objective="reg:logistic", max_depth = params["max_depth"], gamma = params["gamma"], eta = params["eta"], use_label_encoder=False)
    model.fit(x_train, y_train)
    models[run_name] = model
    y_pred_proba = model.predict_proba(x_val)[:, 1]
    y_pred = model.predict(x_val)
    acc_score = accuracy_score(y_val, y_pred)
    val_f1_score = f1_score(y_val, y_pred, average="weighted")
    vertex_ai.log_metrics({"acc_score": acc_score, "f1score": val_f1_score})
    vertex_ai.end_run()

We know that XGBoost usually performs well on imbalanced datasets. But lets also try another algorithm, such as logistic regression, to see how various runs looks like in experiment:

In [ ]:
parameters = [
    {"solver": "lbfgs", "penalty": "none"},
    {"solver": "lbfgs", "penalty": "l2"},
    {"solver": "liblinear", "penalty": "l2"},
    {"solver": "liblinear", "penalty": "l1"},
]

for i, params in enumerate(parameters):
    run_name=f"fd-logisticreg-local-run-{i}"
    print(run_name)
    vertex_ai.start_run(run=run_name)
    vertex_ai.log_params(params)
    clf = LogisticRegression(random_state=0, penalty=params["penalty"], solver = params["solver"]).fit(x_train, y_train.array)
    y_pred = clf.predict(x_val[:, :])
    clf.predict_proba(x_val[:2, :])
    models[run_name] = clf
    acc_score = clf.score(x_val, y_val)
    val_f1_score = f1_score(y_val, y_pred, average="weighted")
    vertex_ai.log_metrics({"acc_score": acc_score, "f1score": val_f1_score})
    vertex_ai.end_run()

We can also extract all parameters and metrics associated with any experiment into a dataframe for further analysis.

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df.head(5)

Also we can visualize experiments using the Cloud Console. Run the following cell to get the URL of [Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments) for your project. You can use follow the URL to see your results in the Cloud Console.

In [ ]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project={PROJECT_ID}"
)

#### Save model
Now that we have run our experiments, lets choose one of the experiments, and use `xgboost.Booster` `save_model` method to export the model to a local file named `model.bst`.

In [ ]:
model_directory = "../models"
!sudo mkdir -p -m 777 {model_directory}

model = models["fd-xgboost-local-run-0"]
artifact_filename = "model.bst"
model_path = os.path.join(model_directory, artifact_filename)
model.save_model(model_path)

## Model Evaluation

Let's first test the model locally on your testset, to get predicted labels and an F1 score, which is an aggregation of the model's precision and recall:

In [ ]:
bst = xgb.Booster()  # init model
bst.load_model(model_path) 
xgtest = xgb.DMatrix(x_test)
y_pred_prob = bst.predict(xgtest)
y_pred = y_pred_prob.round().astype(int)
y_pred

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test.values, y_pred, average="weighted")

The decision to convert a predicted probability into a fraudulant/non-fradulant class label is determined by a discrimination threshold, which uses a default value of 0.5. That is, a transaction is predicted as non-fraudulant (class 0) if the probability is under 0.5, and fraudulant (class 1) if it is equal to or greater than 0.5. This threshold determines the True Positive, False Pasitive, True Negative, and False Negative results which are typically used in confusion matrix, precision, recall, and F1-score, all of which are used as accuracy metrics for a classification model.

You might get different TP and FP rates if you change this threshold, especially if your data is unbalanced. By fine-tuning this threshold, you might find a value that leads to near optimal model performance, based on your business tolerance for accepting the cost of FP or FN cases. In the following cell we calculate the confusion matrix for different discrimination thresholds:

In [ ]:
fig, axes = plt.subplots(2,5, figsize=(15,4))
     
for n, ax in enumerate(axes.flat):
    threshold =  (n+1)/10
    y_pred = (y_pred_prob > threshold).astype(int)
    cfm = metrics.confusion_matrix(y_test, y_pred)
    sns.heatmap(cfm, annot=True, cmap="Reds", fmt="d", ax=ax, cbar=False)
    ax.title.set_text("Threshold=%.2f" % threshold)
  
plt.subplots_adjust(hspace=0.4, wspace=0.8)
plt.suptitle("Confusion Matrix for various thresholds")

We might get more insight into the optimal threshold by examining a Receiver Operator Characteristic (ROC) Curve. This will plot the true positive (TP) vs. false positive (FP) rates at different classification thresholds. Lowering the classification threshold classifies more items as positive, thus increasing both False Positives and True Positives. The following figure shows a typical ROC curve. Also, we graph the Area Under the Curve (AUC) which ranges in value from 0 to 1. A model whose predictions are 100% wrong has an AUC of 0.0; one whose predictions are 100% correct has an AUC of 1.0.

In [ ]:
precision_recall_fscore_support(y_test.values, y_pred, average="weighted")
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, y_pred_prob)
auc = metrics.auc(fpr, tpr)
  
plt.figure(figsize=(8, 6))
# plot the roc curve for the model
plt.plot(fpr, tpr, marker=".",label="xgboost: AUC = %.2f" % auc)

# generate general prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
ns_fpr, ns_tpr, _ = metrics.roc_curve(y_test, ns_probs)
ns_auc = metrics.auc(ns_fpr, ns_tpr)
plt.plot(ns_fpr, ns_tpr, linestyle="--", label="No Skill: AUC = %.2f" % ns_auc)
  
plt.ylabel("TP rate")
plt.xlabel("FP rate")
  
plt.legend(loc=4)
plt.title("ROC Curve")
plt.show()

So, what is the optimal threshold? As mentioned, this depends on the business use case, for example, if we believe the optimal threshold for our use case is  one that offers the highest TPR and minimum FPR, then we can calculate the optimal threshold as follows:

In [ ]:
print("Optimal threshold value is: %.3f" % thresholds[np.argmax(tpr - fpr)])

Now that we understand our dataset more deeply, have achieved good results with the XGBoost algorithm, and have fine-tuned model hyperparameters, let's turn our attention in the next notebook to transitioning this training process from an ad hoc approach to a more formal method.

You can continue with the next Notebook: `04_model_training_xgboost_formalization.ipynb`.